### 1. Import libraries

In [1]:
import pandas as pd
import requests
import json
import os
import shutil
from tqdm.auto import tqdm
import zipfile
import glob
import re
import pyarrow as pa
import rpy2_arrow.pyarrow_rarrow as pyra
import pyarrow.dataset as ds
import gc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


### 2. Download the data

In [2]:
# Metadata
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
out_dir = os.path.join(os.getcwd(), "..", "data", "raw", "figshare")
file_to_download = "data.zip"

# Get file url
file_url = [
    item_["download_url"]
    for item_ in requests.get(url, headers=headers).json()["files"]
    if item_["name"] == file_to_download
][0]

# Check if file has already been downloaded
if os.path.exists(os.path.join(out_dir, file_to_download)):
    print("File already exists. Skipping.")
else:
    print(f"Writing file file {file_to_download} to directory {out_dir}")

    # Create an HTTP request
    with requests.get(file_url, stream=True) as r:

        # Check content length
        content_length = int(r.headers.get("Content-Length"))

        # SDisplay progress bar
        with tqdm.wrapattr(r.raw, "read", total=content_length, desc="") as raw:

            # Save file
            os.makedirs(out_dir)
            with open(os.path.join(out_dir, 
                                   file_to_download), "wb") as path:
                shutil.copyfileobj(raw, path)

    print("Download complete.")

if not any(fname.endswith('.csv') for fname in os.listdir('.')):
    # Unzip file with python
    print("Unzipping file...")
    with zipfile.ZipFile(os.path.join(out_dir, file_to_download), "r") as zip_ref:
        zip_ref.extractall(out_dir) # Extract all files to directory
        zip_ref.close()
    print("Unzipping complete.")


File already exists. Skipping.
Unzipping file...
Unzipping complete.


### 3. Combining data csv

In [3]:
out_processed_dir = os.path.join(os.getcwd(), "..", "data", "processed", "figshare")
file_to_exclude = "observed_daily_rainfall_SYD.csv"
files = glob.glob(out_dir + "/*.csv")

In [4]:
%%timeit -r 1

# Combine data
df = pd.concat(
    (
        pd.read_csv(file, index_col=0).assign(model=re.findall(r"[^\/]+(?=\_daily)", os.path.basename(file))[0])
        for file in files
        if file_to_exclude not in file
    )
)

# Write to file
os.makedirs(out_processed_dir, exist_ok=True)  
df.to_csv(os.path.join(out_processed_dir, "processed_rainfall.csv"))


4min 56s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


#### Compare run times on different machines - Combining data

| Team Member        | Operating System | RAM  | Processor              | Is SSD | Time taken |
|:------------------:|:----------------:|:----:|:----------------------:|:------:|:----------:|
| Rakesh Pandey      | Ubuntu 20.04     | 32GB | Intel® Core™ i7-10870H | Yes    | 4min 51s   |
| Mahsa Sarafrazi    | Windows 11 64-bit| 8 GB | Intel® Core™ i5-1035G4 |Yes     | 17min 4s   |
| Gabe Fairbrother   |  Windows 10      | 32GB | Intel® Core™ i7-10875H | Yes    |     6min 40s       |
| Michelle Wang      | Windows 10        | 16GB | Intel® Core™ i5-11300H  |      Yes                  |  15min 25s    

### 4. Load the combined CSV to memory and perform a simple EDA

#### A. Load all columns

In [5]:
gc.collect()

0

In [6]:
%%timeit -r 1

# Load the data
df = pd.read_csv(os.path.join(out_processed_dir, "processed_rainfall.csv"), index_col=0)

# Get the model counts
print("Model counts:")
print(df.model.value_counts())

# Describe the data
print("Data description:")  
print(df.describe())



Model counts:
MPI-ESM1-2-HR       5154240
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
Data description:
            lat_min       lat_max       lon_min       lon_max  rain (mm/day)
count  5.924854e+07  6.246784e+07  5.924854e+07  6.246784e+07   5.924854e+07
mean  -3.310482e+01 -3.197757e+0

#### Compare run times on different machines - Load all columns

| Team Member        | Operating System | RAM  | Processor              | Is SSD | Time taken |
|:------------------:|:----------------:|:----:|:----------------------:|:------:|:----------:|
| Rakesh Pandey      | Ubuntu 20.04     | 32GB | Intel® Core™ i7-10870H | Yes    | 1min 0s   |
| Mahsa Sarafrazi    | Windows 11 64-bit   | 8 GB | Intel® Core™ i5-1035G4 | Yes    | 3min 37s  |
| Gabe Fairbrother   |  Windows 10      | 32GB | Intel® Core™ i7-10875H | Yes    |   1min 18s       |
| Michelle Wang      |  Windows 10        | 16GB | Intel® Core™ i5-11300H  |   Yes |  3min 29s          |

#### B. Load only required columns


In [7]:
gc.collect()

0

In [8]:
%%timeit -r 1
use_cols = ["time", "rain (mm/day)", "model"]
df = pd.read_csv(
    os.path.join(out_processed_dir, "processed_rainfall.csv"),
    index_col=0,
    parse_dates=True,
    usecols=use_cols,
)

# Get the model counts
print("Model counts:")
print(df.model.value_counts())

# Describe the data
print("Data description:")
print(df.describe())


Model counts:
MPI-ESM1-2-HR       5154240
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
Data description:
       rain (mm/day)
count   5.924854e+07
mean    1.901170e+00
std     5.585735e+00
min    -3.807373e-12
25%     3.838413e-06
50%     6.154947e-02
75%     1.020918e+00
max     4.329395e+

#### Compare run times on different machines - Load only required cols

| Team Member        | Operating System | RAM  | Processor              | Is SSD | Time taken |
|:------------------:|:----------------:|:----:|:----------------------:|:------:|:----------:|
| Rakesh Pandey      | Ubuntu 20.04     | 32GB | Intel® Core™ i7-10870H | Yes    | 46.8s      |
| Mahsa Sarafrazi    | Windows 64-bit   | 8 GB | Intel® Core™ i5-1035G4 | Yes    | 7min 34s  |
| Gabe Fairbrother   |  Windows 10      | 32GB | Intel® Core™ i7-10875H | Yes    |    1min 26s      |
| Michelle Wang      | Windows 10        | 16GB | Intel® Core™ i5-11300H  |   Yes |   3min 15s         |

> We can see that time is now slightly reduced: loading required columns reduced time taken for most of us - 3 out of 4 members (previously from about 1 minutes+ to now under 1 minute).

#### C. Change dtype and use only required columns

In [9]:
%%timeit -r 1

use_cols = ["time", "rain (mm/day)", "model"]
dtypes = {"rain (mm/day)": "float32", "model": "str"}

df = pd.read_csv(
    os.path.join(out_processed_dir, "processed_rainfall.csv"),
    index_col=0,
    parse_dates=True,
    usecols=use_cols,
    dtype=dtypes,
)

# Get the model counts
print("Model counts:")
print(df.model.value_counts())

# Describe the data
print("Data description:")
print(df.describe())



Model counts:
MPI-ESM1-2-HR       5154240
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
Data description:
       rain (mm/day)
count   5.924854e+07
mean    1.901173e+00
std     5.585735e+00
min    -3.807373e-12
25%     3.838413e-06
50%     6.154947e-02
75%     1.020918e+00
max     4.329395e+

#### Compare run times on different machines - Change dtype and load required cols

| Team Member        | Operating System | RAM  | Processor              | Is SSD | Time taken |
|:------------------:|:----------------:|:----:|:----------------------:|:------:|:----------:|
| Rakesh Pandey      | Ubuntu 20.04     | 32GB | Intel® Core™ i7-10870H | Yes    | 46.1s      |
| Mahsa Sarafrazi    | Windows 11 64-bit   | 8 GB | Intel® Core™ i5-1035G4 | Yes    | 9min 55s   |
| Gabe Fairbrother   |  Windows 10      | 32GB | Intel® Core™ i7-10875H | Yes    |    1min 21s|
| Michelle Wang      |  Windows 10        | 16GB | Intel® Core™ i5-11300H  |   Yes |   2min 58s         |

> Adding onto the above, changing dtype has further reduced our time for most of us (3 out of 4 members) slightly. 

#### D. Use chunks

In [10]:
gc.collect()

3

In [11]:
%%timeit -r 1

df = pd.DataFrame()
counts = pd.Series(dtype=int)

for chunk in pd.read_csv(
    os.path.join(out_processed_dir, "processed_rainfall.csv"),
    index_col=0,
    parse_dates=True, 
    chunksize=1_000_000):
    df = pd.concat([df, chunk])
    

# Get the model counts
print("Model counts:")
print(df.model.value_counts())

# Describe the data   
print("Data description:")
print(df.describe())

Model counts:
MPI-ESM1-2-HR       5154240
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
Data description:
            lat_min       lat_max       lon_min       lon_max  rain (mm/day)
count  5.924854e+07  6.246784e+07  5.924854e+07  6.246784e+07   5.924854e+07
mean  -3.310482e+01 -3.197757e+0

#### Compare run times on different machines - Chunking

| Team Member        | Operating System | RAM  | Processor              | Is SSD | Time taken |
|:------------------:|:----------------:|:----:|:----------------------:|:------:|:----------:|
| Rakesh Pandey      | Ubuntu 20.04     | 32GB | Intel® Core™ i7-10870H | Yes    | 1min 34s   |
| Mahsa Sarafrazi    | Windows 64-bit   | 8 GB | Intel® Core™ i5-1035G4 | Yes    | 5min 44s   | 
| Gabe Fairbrother   |  Windows 10      | 32GB | Intel® Core™ i7-10875H | Yes    |     2min 12s      |
| Michelle Wang      |  Windows 10        | 16GB | Intel® Core™ i5-11300H  |   Yes  |    3min 50s        |

> For chunking, it seems like there is not much improvement for most of us.

**EDA Python Conclusion:**
After trying out the above techniques, we can conclude that both loading the required columns and changing datatypes are effective ways of reducing runtime.

> **Note**
> The below are some visualization EDAs that are commented out due to 2 main reasons:
> 1. reading the whole data and working with the pandas library, makes the computer crash. Using a small part of the data and making a conclusion is misleading, as it cannot give an accurate insight from the whole data.
> 2. reading a small part of data(1_000_000 rows) would increase the size of NB to 200MB which is cannot be uploaded on Github.
> As a result, all EDA codes are available in the below cells but we recommend not to run them.

**Visualization plots on subsampled data**


##### Reading the Dataframe:

In [12]:
# %%timeit -r 1
# df = pd.read_csv(
#     os.path.join(out_processed_dir, "processed_rainfall.csv"),
#     index_col=0,
#     usecols=["time", "rain (mm/day)", "model"],
#     parse_dates=True,
# )
# df.head()

##### Sampling 1_000_000 rows

In [13]:
# df_sample = df.sample(n=1000000, random_state=42)
# df_sample.to_csv(os.path.join(out_processed_dir, "EDA.csv"))

##### Rainfall distribution:

In [14]:
# %%timeit -r 1
# plot = (
#     alt.Chart(df_sample, title="Total rain distribution")
#     .mark_boxplot(extent="min-max")
#     .encode(alt.X("rain (mm/day)"))
# )
# plot

##### Rainfall distribution based on model:

In [15]:
# %%timeit -r 1
# plot = (
#     alt.Chart(df_sample, title="Rain distribution based on model")
#     .mark_boxplot(extent="min-max")
#     .encode(
#         alt.X("rain (mm/day)"),
#         alt.Y(
#             "model",
#             sort=alt.EncodingSortField(
#                 field="rain (mm/day)", op="median", order="descending"
#             ),
#         ),
#         color="model",
#     )
# )
# plot

#### Rainfall histogram based on model

In [16]:
# %%timeit -r 1
# alt.data_transformers.disable_max_rows()
# plot_hist = (
#     alt.Chart(df_sample, title="Rain fall histogram based on model")
#     .mark_bar()
#     .encode(alt.X("rain (mm/day)"), alt.Y("count():Q"), color="model")
#     .properties(width=180, height=180)
#     .facet(facet="model", columns=9)
# )
# plot_hist

### 6. Perform a simple EDA in R

We shall try out a few methods with simple EDA to test the efficiency of each method to convert data into R formats: Parquet file, Feather and Arrow. Then we compare the run times for each of them and see which ones are faster.

In [17]:
# Create df with only the model column

out_processed_dir = os.path.join(os.getcwd(), "..", "data", "processed", "figshare")
use_cols = ["model"]
dtypes = {"model": "str"}

df = pd.read_csv(
    os.path.join(out_processed_dir, "processed_rainfall.csv"),
    index_col=0,
    parse_dates=True,
    usecols=use_cols,
    dtype=dtypes,
)

#### a) Parquet file method

In [18]:
%%time

if os.path.exists(os.path.join(out_processed_dir, "rainfall.parquet")):
    print("Parquet File already exists. Skipping.")
else:
    df.to_parquet(os.path.join(out_processed_dir, "rainfall.parquet"))

Parquet File already exists. Skipping.
CPU times: user 320 µs, sys: 22 µs, total: 342 µs
Wall time: 247 µs


In [19]:
%reload_ext rpy2.ipython

In [20]:
%%time
%%R
suppressMessages(library(arrow, warn.conflicts = FALSE))
suppressMessages(library(dplyr, warn.conflicts = FALSE))
library(here)

ds <- open_dataset(here("data/processed/figshare/rainfall.parquet"))
result <- ds %>% count(model, sort=TRUE)

# My windows comp crashes for this line
# print(result %>% collect)

R[write to console]: here() starts at /home/veerubhai/Videos/BLOCK_6/525/dsci_525_group_22



CPU times: user 248 ms, sys: 43.2 ms, total: 292 ms
Wall time: 328 ms


> To convert pandas df to parquet, it took 22.6s and then loading the 'ds' parquet file was super fast because it hasn't processed anything.  
> Note: The last line seems to crash the computer so it's commented out.

#### b) Feather method

In [21]:
%%time

# Create feather file from pandas df
if os.path.exists(os.path.join(out_processed_dir, "rainfall.feather")):
    print("Feather File already exists. Skipping.")
else:
    df.reset_index().to_feather(os.path.join(out_processed_dir, "rainfall.feather"))

Feather File already exists. Skipping.
CPU times: user 209 µs, sys: 0 ns, total: 209 µs
Wall time: 134 µs


> Converting pandas df to feather file took 8.5s. 

#### c) Arrow method

In [22]:
# NOTE: This code crashes for windows comp. Using the next cell instead.
# %%time

# dataset = ds.dataset(os.path.join(out_processed_dir, "processed_rainfall.csv"), format="csv")
# table = dataset.to_table()
# r_table = pyra.converter.py2rpy(table)

In [23]:
%%time
rdf = pyra.converter.py2rpy(pa.Table.from_pandas(df))

CPU times: user 1.13 s, sys: 388 ms, total: 1.52 s
Wall time: 1.5 s


In [24]:
%%time
%%R -i rdf
library(dplyr)

# Get the model counts
result <- rdf %>% count(model, sort=TRUE)
print(result %>% collect())

# Describe the data
print("Data description:")
print(summary(rdf))


# A tibble: 27 × 2
   model               n
   <chr>           <int>
 1 MPI-ESM1-2-HR 5154240
 2 CMCC-CM2-SR5  3541230
 3 NorESM2-MM    3541230
 4 CMCC-ESM2     3541230
 5 CMCC-CM2-HR4  3541230
 6 TaiESM1       3541230
 7 SAM0-UNICON   3541153
 8 FGOALS-f3-L   3219300
 9 GFDL-ESM4     3219300
10 GFDL-CM4      3219300
# … with 17 more rows
[1] "Data description:"
      Length   Class        Mode       
model 62467843 ChunkedArray environment
CPU times: user 2.25 s, sys: 90.9 ms, total: 2.34 s
Wall time: 277 ms


> Converting pandas df to arrow table object is fast: only 7.5s. Then printing the results of count by models and summary of dataset was only around 2.5s.

> Note: The above Arrow Exchange conclusions are based on runtimes used by a teammate using a different processor, which differs from current output.

**Final chosen approach: Arrow exchange**

- After experimenting with different conversion methods to R, we concluded that the 'Arrow Exchange' method works best in terms of speed and implementation. 
- With parquet method, it took 26s just to convert the pandas file to parquet. 
- With feather file method, it took 8.5s. 
- The fastest was Arrow which took around 7s. 

Arrow exchange is the best in terms of timing and for the following reasons:
- The pyarrow package uses compiled code to efficiently convert a `pandas DataFrame` to an `Arrow` data structure, and the R arrow package can do the same from a `Arrow` data structure to a `R data.frame`.
- The `arrow` table structure is also well-integrated with R's Dplyr package functionalities and makes EDA extremely fast and convenient, as exemplified in the code above where the printing of EDA results only took 2s.
- Time spent on arrow's serialization/deserialization process is minimal and is also a zero-copy process.

### 